In [945]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [201]:
!pip install node2vec

In [202]:
from node2vec import Node2Vec

In [946]:
# Loads the saved pickle file data

file = open('./Pickle Files/Data_Wrangling_Data.pickle','rb')
fb = pickle.load(file)
G = pickle.load(file)
node_attr = pickle.load(file)

# Pre-Processing For Node2Vec Link Prediction Approach

- The goal is to generate features for the model to be able to use with a supervised learning model to predict if a pair of nodes are connected or not
- The Node2Vec algorithm generates features for a node and needs to learn how to generate those features
- To do this, Node2Vec needs to understand a previous version of the graph to generate the appropriate features for new connected or unconnected edges

Before Deleting Removable Edges
<img src = 'Images/before.png' style='width:200px;height:200px'> After Deleting Removable Edges, Dotted Edges are removable
<img src = 'Images/after.png' style='width:200px;height:200px'>  
1. Find the negative samples (unconnected edges / node pairs) with nodes at max path length of 2 from one another to get samples that are possibly more likely to form a connection due to more common neighbors
2. Find the positive samples (removable edges / node pairs) - edges that can be removed while preserving the base graph structure (not eliminating nodes & not splitting the graph)
3. Find the remaining edges by removing the positive edges from the list of connected edges so we can train the Node2Vec Model on those remaining edges (base structure of the graph)
4. Combine negative & positive samples into final dataframe after reducing the # of negative samples to balance final dataset to an about equal positive & negative samples to be used for more accurate modeling
5. Use the trained Node2Vec model to generate features for each node in a pair & sum them for a final set of features for that pair

In [163]:
# Creates a adjacency matrix of node pairs
adj_matrix = nx.to_numpy_matrix(G)
print(adj_matrix.shape)
adj_matrix

(4039, 4039)


matrix([[0., 1., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [164]:
# Finds Negative Samples
# Finds and creates a dataframe of node pairs that do not have a edge between them and gets a subset of them to lower the amount of data to deal with

from tqdm import tqdm 

offset = 0
unconnected_node_pairs = []

# Loops through the adjaceny matrix - top half above the diagonal and checks if those node pairs are not connected
for i in tqdm(range(len(adj_matrix))):
    for j in range(offset, len(adj_matrix)):
        # Finds node pairs that are not connected and have a path within a length of 2
        if i != j and adj_matrix[i,j] == 0 and nx.has_path(G, i, j) and nx.shortest_path_length(G, i, j) <= 2:
            unconnected_node_pairs.append([i, j])
    offset += 1 

# Creates a dataframe of unconnected edges
unconnected_df = pd.DataFrame(data = unconnected_node_pairs, columns = ['Node 1', 'Node 2'])
unconnected_df['Connected'] = 0

print(f'Unconneted Node Pairs: {len(unconnected_df.index)}')
unconnected_df

100%|██████████| 4039/4039 [1:08:26<00:00,  1.02s/it]  


Unconneted Node Pairs: 1395922


,Node 1,Node 2,Connected
0,0,348,0
1,0,351,0
2,0,353,0
3,0,363,0
4,0,364,0
...,...,...,...
1395917,4035,4037,0
1395918,4035,4038,0
1395919,4036,4037,0
1395920,4036,4038,0


In [839]:
# Reduces the list of unconnected edges to balance the positive and negative pairs in the data

while True:
    
    new_unconnected = sorted(random.sample(unconnected_node_pairs, 85000))
    new_unconnected_df = pd.DataFrame(data = new_unconnected, columns = ['Node 1', 'Node 2'])
   
    a = list(new_unconnected_df.loc[:,'Node 1'].unique())
    b = list(new_unconnected_df.loc[:,'Node 2'].unique())
    a.extend(b)
    
    if not (len(set(a)) == len(G.nodes)): continue
    else: break
    
# Creates a dataframe of unconnected edges
new_unconnected_df['Connected'] = 0

print(f'Unconneted Node Pairs: {len(new_unconnected_df.index)}')
print(new_unconnected_df)
%store new_unconnected_df

Unconneted Node Pairs: 85000
       Node 1  Node 2  Connected
0           0     364          0
1           0     906          0
2           0     961          0
3           0     970          0
4           0     978          0
...       ...     ...        ...
84995    4024    4036          0
84996    4028    4034          0
84997    4029    4038          0
84998    4030    4031          0
84999    4034    4036          0

[85000 rows x 3 columns]
Stored 'new_unconnected_df' (DataFrame)


In [921]:
# Positive Samples - Removable Edges (Subset Of Connected Node Pairs)
# Finds removable edges / node pairs - edges that can be removed without changing the graph structure
# We will remove these edges from the list of connected edges so we can train the Node2Vec Model on those remaining edges

temp_df = fb.copy()
removable_node_pairs_indices = []

# Iterates through the dataframe of connected node pairs
for i in tqdm(fb.index.values):
    test_G = nx.from_pandas_edgelist(temp_df.drop(index = i), 'Node 1', 'Node 2')
    
    # Makes sure the graph is not being split into more unconnected graphs than the original graph
    # If it is not being split, considers it a removable edge
    if (nx.number_connected_components(test_G) == nx.number_connected_components(G)) and (len(nx.nodes(test_G)) == len(nx.nodes(G))):
        # Keeps track of indices in connected node pairs data frame where edge is removable
        removable_node_pairs_indices.append(i)
        # Dataframe of connected node pairs with removed removable edges used to train Node2Vec model
        temp_df = temp_df.drop(index = i)

# Creates dataframe of removable edges / node pairs 
# Removable edges are connected so will recieve a 1 in the connection column when training a ML model
removable_df = fb.loc[removable_node_pairs_indices]
removable_df['Connected'] = 1
removable_df.reset_index(drop = True)

print(f'Removable Node Pairs: {len(removable_df.index)}')
removable_df

100%|██████████| 88234/88234 [4:44:43<00:00,  5.16it/s]      


Removable Node Pairs: 84196


,Node 1,Node 2,Connected
0,0,1,1
1,0,2,1
2,0,3,1
3,0,4,1
4,0,5,1
...,...,...,...
88219,4020,4030,1
88220,4020,4031,1
88223,4021,4026,1
88226,4023,4031,1


In [922]:
# Remaining Edges - Creates the node2vec_df (dataframe of connected node pairs with removed removable edges), to train Node2Vec model to generate features

node2vec_df = fb.copy()

for i in removable_node_pairs_indices:
    node2vec_df = node2vec_df.drop(index = i)

node2vec_df.reset_index(inplace = True) 
node2vec_df.drop(columns = ['index'], inplace = True)
print(f'Node2Vec Edges: {len(node2vec_df.index)}')
node2vec_df

Node2Vec Edges: 4038


,Node 1,Node 2
0,0,11
1,0,12
2,0,15
3,0,18
4,0,37
...,...,...
4033,4023,4038
4034,4026,4030
4035,4027,4032
4036,4027,4038


In [923]:
# Creates 'final_df' dataframe (unconnected pairs + removable pairs/edges) for training and predicting ML models
# The removable edges and unconncted edges will be used when training and testing with a ML model

final_df = new_unconnected_df.append(removable_df, ignore_index = True)

print(f'Unconnected & Removable Edges: {len(final_df.index)}')
print(final_df['Connected'].value_counts())
final_df

Unconnected & Removable Edges: 169196
0    85000
1    84196
Name: Connected, dtype: int64


,Node 1,Node 2,Connected
0,0,364,0
1,0,906,0
2,0,961,0
3,0,970,0
4,0,978,0
...,...,...,...
169191,4020,4030,1
169192,4020,4031,1
169193,4021,4026,1
169194,4023,4031,1


In [924]:
# Creates and fits Node2Vec model 'node2vec_G' on 'node2vec_df' (Connected node pairs after removing removable edges)

node2vec_G = nx.from_pandas_edgelist(node2vec_df, 'Node 1', 'Node 2')
nx.set_node_attributes(node2vec_G, node_attr)
node2vec_model = Node2Vec(node2vec_G, dimensions=100, walk_length=16, num_walks=50).fit(window=7, min_count=1)
print(nx.info(node2vec_G))
print(node2vec_model)

%store node2vec_model

Generating walks (CPU: 1):   0%|          | 0/50 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 50/50 [01:56<00:00,  2.33s/it]


Name: 
Type: Graph
Number of nodes: 4039
Number of edges: 4038
Average degree:   1.9995
Word2Vec(vocab=4039, vector_size=100, alpha=0.025)
Stored 'node2vec_model' (Word2Vec)


# Creates X, y and training and testing sets 

In [925]:
# For node2vec, X = Edge Features of final_df (Unconnected & Removable Edges), y = final_df['Connected']

X = []
for i,j in zip(final_df['Node 1'], final_df['Node 2']):
    X.append(node2vec_model.wv[str(i)] + node2vec_model.wv[str(j)])

X = np.array(X)
y = final_df['Connected']

print(f'Features (X): {X.shape}')
print(X)

Features (X): (169196, 100)
[[ 0.05423658  0.38881892 -0.7825276  ... -0.3608516   0.13439018
  -0.14539672]
 [-0.06878684  0.9884709  -0.85741556 ... -0.45005986 -0.20884675
  -0.87278837]
 [-0.4779037   0.9238887  -1.0211918  ... -0.7530286   0.5291232
  -0.5641365 ]
 ...
 [-0.29917532  0.92449725 -1.1229932  ... -0.36626023 -0.29428327
  -0.489528  ]
 [ 0.3246755   1.0931772  -0.41049516 ... -0.01053643 -0.36503953
   0.14392054]
 [ 0.48538733  0.91636264 -0.595041   ...  0.15409005 -0.65976167
   0.33824128]]


In [926]:
# Creates training and testing data for supervised learning models

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

%store X_train
%store X_test
%store y_train
%store y_test 

Stored 'X_train' (ndarray)
Stored 'X_test' (ndarray)
Stored 'y_train' (Series)
Stored 'y_test' (Series)


In [956]:
# Save the train and test data to pickle file

file = open('./Pickle Files/Node2Vec_Pre-Processing_Data.pickle','wb')
pickle.dump(new_unconnected_df, file)
pickle.dump(node2vec_model, file)
pickle.dump(X_train, file)
pickle.dump(X_test, file)
pickle.dump(y_train, file) 
pickle.dump(y_test, file)

# Pre-Processing For NetworkX Link Prediction Approach

The way to recommend Facebook Friends using this approach is as follows: 

- Use the NetworkX library link prediction algorithms to calculate the link metric for each pair in the list of unconnected (negative samples) & removable edges (positive samples) to generate features that will be applied to a supervised learning approach to predict if a pair of nodes are connected or not 
- I will use the following link prediction algorithms: common_neighbors, jaccard_coefficient, resource_allocation index, adamic_adar_index, preferential_attachment
- Create a function that goes through all the unconnected pairs including the specific user picked to recommend a friend for and recommend top k users with which the selected user has the highest probabilties of forming a connection with

In [927]:
final_df_nx = final_df.copy()

In [928]:
# Gets the pairs of nodes as a list of tuples to be used

node_pairs = []
for i in tqdm(range(len(final_df_nx.index))):
    node_pairs.append((final_df_nx.iloc[i]['Node 1'], final_df_nx.iloc[i]['Node 2']))

100%|██████████| 169196/169196 [00:26<00:00, 6499.97it/s]


# Common Neighbors
https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.classes.function.common_neighbors.html

In [929]:
common_neighbors = []
for n in node_pairs:
    common_neighbors.append(len(list(nx.common_neighbors(G, n[0], n[1]))))
final_df_nx['Common Neighbors'] = [i for i in common_neighbors]

# Jaccard Coefficient 

https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_prediction.jaccard_coefficient.html#networkx.algorithms.link_prediction.jaccard_coefficient

In [930]:
jaccard_coefficient = list(nx.jaccard_coefficient(G, node_pairs))
final_df_nx['Jaccard Coefficient'] = [i[2] for i in jaccard_coefficient]

# Resource Allocation Index
https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_prediction.resource_allocation_index.html#networkx.algorithms.link_prediction.resource_allocation_index

In [931]:
resource_allocation_index = list(nx.resource_allocation_index(G, node_pairs))
final_df_nx['Resource Allocation Index'] = [i[2] for i in resource_allocation_index]

# Adamic Adar Index
https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_prediction.adamic_adar_index.html#networkx.algorithms.link_prediction.adamic_adar_index

In [932]:
adamic_adar_index = list(nx.adamic_adar_index(G, node_pairs))
final_df_nx['Adamic Adar Index'] = [i[2] for i in adamic_adar_index]

# Preferential Attachment
https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_prediction.preferential_attachment.html#networkx.algorithms.link_prediction.preferential_attachment

In [933]:
preferential_attachment = list(nx.preferential_attachment(G, node_pairs))
final_df_nx['Preferential Attachment'] = [i[2] for i in preferential_attachment]

In [934]:
final_df_nx

,Node 1,Node 2,Connected,Common Neighbors,Jaccard Coefficient,Resource Allocation Index,Adamic Adar Index,Preferential Attachment
0,0,364,0,1,0.002817,0.083333,0.402430,3123
1,0,906,0,1,0.002358,0.000957,0.143848,27066
2,0,961,0,1,0.002786,0.000957,0.143848,4511
3,0,970,0,1,0.002770,0.000957,0.143848,5205
4,0,978,0,1,0.002222,0.000957,0.143848,36088
...,...,...,...,...,...,...,...,...
169191,4020,4030,1,2,0.080000,0.116949,0.679541,152
169192,4020,4031,1,5,0.357143,0.513775,2.162457,88
169193,4021,4026,1,6,0.428571,0.474343,2.329151,99
169194,4023,4031,1,5,0.208333,0.537584,2.206669,198


# Creates X, y and training and testing sets 

In [935]:
# For NetworkX link prediction, X = Link Prediction Algo. Values of final_df_link_vals (Unconnected & Removable Edges), y = final_df_link_vals['Connected']

columns = ['Common Neighbors', 'Jaccard Coefficient','Resource Allocation Index','Adamic Adar Index', 'Preferential Attachment']

X_nx = final_df_nx.loc[:,columns]
y_nx = final_df_nx['Connected']

print(f'Features (X): {X_nx.shape}')
print(X_nx)

Features (X): (169196, 5)
        Common Neighbors  Jaccard Coefficient  Resource Allocation Index  \
0                      1             0.002817                   0.083333   
1                      1             0.002358                   0.000957   
2                      1             0.002786                   0.000957   
3                      1             0.002770                   0.000957   
4                      1             0.002222                   0.000957   
...                  ...                  ...                        ...   
169191                 2             0.080000                   0.116949   
169192                 5             0.357143                   0.513775   
169193                 6             0.428571                   0.474343   
169194                 5             0.208333                   0.537584   
169195                 4             0.285714                   0.395917   

        Adamic Adar Index  Preferential Attachment  
0       

In [936]:
# Creates training and testing data and scales the features for supervised learning models

X_train_nx, X_test_nx, y_train_nx, y_test_nx = train_test_split(X_nx, y_nx, test_size = 0.2, random_state = 0, stratify = y_nx)

# Scales training set
scaler = StandardScaler().fit(X_train_nx)
X_train_nx.loc[:, columns] = scaler.transform(X_train_nx)

# Scales test set
X_test_nx.loc[:, columns] = scaler.transform(X_test_nx)

# Scales entire dataset
final_df_nx_scaled = final_df_nx.copy()
final_df_nx_scaled.loc[:, columns] = scaler.transform(final_df_nx.loc[:, columns])

%store X_train_nx
%store X_test_nx
%store y_train_nx
%store y_test_nx
%store final_df_nx_scaled

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


Stored 'X_train_nx' (DataFrame)
Stored 'X_test_nx' (DataFrame)
Stored 'y_train_nx' (Series)
Stored 'y_test_nx' (Series)
Stored 'final_df_nx_scaled' (DataFrame)


In [937]:
final_df_nx_scaled

,Node 1,Node 2,Connected,Common Neighbors,Jaccard Coefficient,Resource Allocation Index,Adamic Adar Index,Preferential Attachment
0,0,364,0,-0.688733,-0.940612,-0.639678,-0.704201,-0.371761
1,0,906,0,-0.688733,-0.942633,-0.871550,-0.733825,1.547445
2,0,961,0,-0.688733,-0.940750,-0.871550,-0.733825,-0.260503
3,0,970,0,-0.688733,-0.940818,-0.871550,-0.733825,-0.204874
4,0,978,0,-0.688733,-0.943233,-0.871550,-0.733825,2.270625
...,...,...,...,...,...,...,...,...
169191,4020,4030,1,-0.665866,-0.600397,-0.545057,-0.672454,-0.609909
169192,4020,4031,1,-0.597264,0.621218,0.571919,-0.502564,-0.615039
169193,4021,4026,1,-0.574396,0.936068,0.460927,-0.483467,-0.614157
169194,4023,4031,1,-0.597264,-0.034718,0.638937,-0.497499,-0.606222


In [938]:
X_train_nx

,Common Neighbors,Jaccard Coefficient,Resource Allocation Index,Adamic Adar Index,Preferential Attachment
129334,-0.071314,1.006034,0.591362,0.047034,-0.460656
95643,-0.437192,-0.099888,-0.016279,-0.388747,-0.525904
48986,-0.688733,-0.926151,-0.870515,-0.733017,-0.583136
42251,-0.688733,-0.895783,-0.871550,-0.733825,-0.504101
32052,-0.002711,0.090060,-0.294699,-0.048955,-0.104196
...,...,...,...,...,...
152792,-0.322855,-0.271809,-0.274126,-0.312083,-0.299860
103443,0.248830,0.955543,1.095760,0.408974,-0.249522
54630,0.774780,0.554934,0.816334,0.821814,0.680464
35936,-0.688733,-0.932809,-0.871550,-0.733825,0.055237


In [958]:
# Save the final, train, and test data to pickle file

file = open('./Pickle Files/NetworkX_Pre-Processing_Data.pickle','wb')
pickle.dump(final_df_nx_scaled, file)
pickle.dump(X_train_nx, file)
pickle.dump(X_test_nx, file)
pickle.dump(y_train_nx, file)
pickle.dump(y_test_nx, file) 